# Machine Learning Notes - Backpropagation

## A List of Related Posts
1. [Pytorch]({% post_url 2021-05-04-machine-learning-pytorch %})
2. [Loss function]({% post_url 2021-05-07-machine-learning-loss %})
3. [Backpropagation(this)]({% post_url 2021-05-07-machine-learning-backpropagation %})

## Backpropagation Introduction

## Chain Rule

$$
y = f_1(x) \\
L = f_2(y)
$$
Chain rule:

$$
\frac{\partial{L}}{\partial{y}} = \frac{\partial{f_2(y)}}{\partial{y}} \\
\frac{\partial{L}}{\partial{x}} = \frac{\partial{L}}{\partial{y}}\frac{\partial{y}}{\partial{x}} = \frac{\partial{f_2(y)}}{\partial{y}} \frac{\partial{f_1(x)}}{\partial{x}}
$$

In [45]:
# Chain Rule

import torch
x = torch.tensor([5], requires_grad=True, dtype=torch.float64)
y = 2 * x
L = 3 * y
dL = 1
dy = 3 * dL
dx = 2 * dy
L.backward()
print(f'dL/dx: {dx}')
print(f'Pytorch autograd {x.grad}')

#

dL/dx: 6
Pytorch autograd tensor([6.], dtype=torch.float64)


$$
\begin{bmatrix}
y_1 \\
y_2
\end{bmatrix} = f(\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix}) = \begin{bmatrix}
f_{1}(x_1, x_2, x_3) \\
f_{2}(x_1, x_2, x_3)
\end{bmatrix}\\
L = g(\begin{bmatrix}
y_1 \\
y_2
\end{bmatrix}) \\ 
$$
Chain rule:

$$
\frac{\partial{L}}{\partial{y}} = \begin{bmatrix} 
    \frac{\partial{g(y)}}{\partial{y_1}} \\ 
    \frac{\partial{g(y)}}{\partial{y_2}} \\ 
\end{bmatrix}\\
\frac{\partial{L}}{\partial{x}} = J_x^y \frac{\partial{L}}{\partial{y}} =
\begin{bmatrix} 
    \frac{\partial{f_1(x)}}{\partial{x_1}}, \frac{\partial{f_2(x)}}{\partial{x_1}}\\ 
    \frac{\partial{f_1(x)}}{\partial{x_2}}, \frac{\partial{f_2(x)}}{\partial{x_2}}\\ 
    \frac{\partial{f_1(x)}}{\partial{x_3}}, \frac{\partial{f_2(x)}}{\partial{x_3}}
\end{bmatrix} \frac{\partial{L}}{\partial{y}}
$$

In [46]:
# Gradients with respect to a vector

# Chain Rule

import torch
x = torch.tensor([1, 2, 3], requires_grad=True, dtype=torch.float64)
y = torch.zeros(2)
y[0] = x[0] + 2*x[1]
y[1] = x[1] + 2*x[2]
L = y.sum()
dL = 1
dy = torch.tensor([1, 1])
dx = torch.tensor([[1, 0],[2, 1], [0, 2]]).matmul(dy)
L.backward()
print(f'dL/dx: {dx}')
print(f'Pytorch autograd {x.grad}')

#

dL/dx: tensor([1, 3, 2])
Pytorch autograd tensor([1., 3., 2.], dtype=torch.float64)


## Local Gradient

... ---> $\textbf{x}$ ---> $f(\textbf{x})$ ---> $\textbf{y}$ ---> ... ---> L

$$
\frac{\partial{L}}{\partial{\textbf{x}}} = J^{\textbf{y}}_\textbf{x}\frac{\partial{L}}{\partial{\textbf{y}}} 
$$
$J^{\textbf{y}}_\textbf{x}$ is a Jacobian matrix, and it has the same number of rows as $\textbf{x}$

## Common Computation Blocks - Forward and Backward Pass

### Vector dot product

Forward pass:
$$
z = x^Ty
$$

Backward pass:
$$
\frac{\partial{L}}{\partial{x}} = \frac{\partial{L}}{\partial{z}}y \\
\frac{\partial{L}}{\partial{y}} = \frac{\partial{L}}{\partial{z}}x
$$
Where $[x]_{nx1}, [y]_{nx1}$

In [47]:
# Vector dot product
import torch
x = torch.tensor([1, 2, 3], requires_grad=True, dtype=torch.float64)
y = torch.tensor([3, 2, 1], requires_grad=True, dtype=torch.float64)
z = x.T.matmul(y)
z.retain_grad()
L = z.sum()
L.backward()
print(f'Pytorch autograd dz: {z.grad}')
print(f'Pytorch autograd dx: {x.grad}')
print(f'Pytorch autograd dy: {y.grad}')

dx = z.grad * y
assert(torch.allclose(dx, x.grad))
dy = z.grad * x
assert(torch.all(torch.isclose(dy, y.grad)))
#

Pytorch autograd dz: 1.0
Pytorch autograd dx: tensor([3., 2., 1.], dtype=torch.float64)
Pytorch autograd dy: tensor([1., 2., 3.], dtype=torch.float64)


### Vector product
Forward pass:
$$
Z = xy
$$

Backward pass:
$$
\frac{\partial{L}}{\partial{x}} = \frac{\partial{L}}{\partial{Z}}y^T \\
\frac{\partial{L}}{\partial{y}} = x^T\frac{\partial{L}}{\partial{Z}}
$$
Where $[x]_{mx1}, [y]_{1xn}, [Z]_{mxn}$


In [48]:
# Vector product

import torch
x = torch.tensor([1, 2, 3], requires_grad=True, dtype=torch.float64).reshape(3,1)
y = torch.tensor([3, 2], requires_grad=True, dtype=torch.float64).reshape(1,2)
z = x.matmul(y)
x.retain_grad()
y.retain_grad()
z.retain_grad()
L = z.sum()
L.backward()
print(f'Pytorch autograd dz:\n {z.grad}')
print(f'Pytorch autograd dx:\n {x.grad}')
print(f'Pytorch autograd dy:\n {y.grad}')

dx = z.grad.matmul(y.T)
assert(torch.allclose(dx, x.grad))
dy = x.T.matmul(z.grad)
assert(torch.all(torch.isclose(dy, y.grad)))
#

Pytorch autograd dz:
 tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], dtype=torch.float64)
Pytorch autograd dx:
 tensor([[5.],
        [5.],
        [5.]], dtype=torch.float64)
Pytorch autograd dy:
 tensor([[6., 6.]], dtype=torch.float64)


### Vector & Matrix product

Forward pass:
$$
Z = Wx
$$

Backward pass:
$$
\frac{\partial{L}}{\partial{x}} = W^T\frac{\partial{L}}{\partial{Z}} \\
\frac{\partial{L}}{\partial{W}} = \frac{\partial{L}}{\partial{Z}}x^T
$$
Where $[x]_{Dx1}, [W]_{CxD}, [z]_{Cx1}$

In [49]:
# Vector product

import torch
x = torch.tensor([1, 2], requires_grad=True, dtype=torch.float64).reshape(2,1)
w = torch.tensor([[1, 2],[2, 1]], requires_grad=True, dtype=torch.float64).reshape(2,2)
z = w.matmul(x)
x.retain_grad()
w.retain_grad()
z.retain_grad()
L = z.sum()
L.backward()
print(f'Pytorch autograd dz:\n {z.grad}')
print(f'Pytorch autograd dx:\n {x.grad}')
print(f'Pytorch autograd dy:\n {w.grad}')

dx = w.T.matmul(z.grad)
assert(torch.all(torch.isclose(dx, x.grad)))

dw = z.grad.matmul(x.T)
assert(torch.allclose(dw, w.grad))
#

Pytorch autograd dz:
 tensor([[1.],
        [1.]], dtype=torch.float64)
Pytorch autograd dx:
 tensor([[3.],
        [3.]], dtype=torch.float64)
Pytorch autograd dy:
 tensor([[1., 2.],
        [1., 2.]], dtype=torch.float64)


### Matrix & Matrix product

Forward pass:
$$
Z = XW
$$

Backward pass:
$$
\frac{\partial{L}}{\partial{X}} = \frac{\partial{L}}{\partial{Z}}W^T \\
\frac{\partial{L}}{\partial{W}} = X^T\frac{\partial{L}}{\partial{Z}}
$$
Where $[x]_{NxD}, [W]_{DxC}, [z]_{NxC}$

In [50]:
# Vector product

import torch
x = torch.tensor([[1, 2, 3], [3, 2, 1]], requires_grad=True, dtype=torch.float64).reshape(2,3)
w = torch.tensor([[1, 2, 3],[2, 1, 3], [3, 1, 2]], requires_grad=True, dtype=torch.float64).reshape(3,3)
z = x.matmul(w)
x.retain_grad()
w.retain_grad()
z.retain_grad()
L = z.sum()
L.backward()
print(f'Pytorch autograd dz:\n {z.grad}')
print(f'Pytorch autograd dx:\n {x.grad}')
print(f'Pytorch autograd dy:\n {w.grad}')

dw = x.T.matmul(z.grad)
assert(torch.all(torch.isclose(dw, w.grad)))

dx = z.grad.matmul(w.T)
assert(torch.allclose(dx, x.grad))
#

Pytorch autograd dz:
 tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
Pytorch autograd dx:
 tensor([[6., 6., 6.],
        [6., 6., 6.]], dtype=torch.float64)
Pytorch autograd dy:
 tensor([[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]], dtype=torch.float64)


### Softmax 

Forward pass:
$$
p_i = \frac{e^{s_i}}{\sum_i{e^{s_i}}} \\
t = \sum_i{e^{s_i}}
$$

Backward pass:
$$
\frac{\partial{L}}{\partial{s}} = J^p_s\frac{\partial{L}}{\partial{p}} \\
J^{p_i}_{s_j} = -p_ip_j \ (i \neq j)\\
J^{p_i}_{s_i} = p_i(1-p_i)
$$
Where $[s]_{Cx1}, [p]_{Cx1}$

In [51]:
# Vector product

import torch
s = torch.tensor([[1, 2]], requires_grad=True, dtype=torch.float64).reshape(2,1)
s_exp = torch.exp(s)
t = torch.sum(s_exp)
p = s_exp / t
s.retain_grad()
p.retain_grad()
# log likelihood
L = -torch.sum(torch.log(p))
L.backward()
print(f'Pytorch autograd dp:\n {p.grad}')
print(f'Pytorch autograd ds:\n {s.grad}')

J = torch.zeros(2, 2, dtype=p.dtype)
J[0, 0] = p[0]*(1-p[0]) # i = 0, i = 0
J[1, 0] = -p[0]*p[1]    # i = 0, j = 1
J[0, 1] = -p[1]*p[0]     # i = 1, j = 0
J[1, 1] = p[1]*(1-p[1]) # i = 0, i = 1
ds = J.matmul(p.grad)

assert(torch.allclose(ds, s.grad))
#

Pytorch autograd dp:
 tensor([[-3.7183],
        [-1.3679]], dtype=torch.float64)
Pytorch autograd ds:
 tensor([[-0.4621],
        [ 0.4621]], dtype=torch.float64)


## Computational Graph